In [1]:
import numpy as np
import pandas as pd
import tqdm
import re
from os.path import join as pathjoin

In [2]:
ru_topics = {
    'music': ['музыка', 'рок-группа', 'альбом', 'песня', 'звук', 'рок', 'гитара', 'джаз', 'рэп', 'музыкант'],
    'education': ['студент', 'обучение', 'курсы', 'изучение', 'навыки', 'образование', 'учитель', 'школа', 'университет'],
    'politics': ['выборы', 'голосование', 'ассамблея', 'кандидат', 'демократия', 
                 'консул', 'министр', 'парламент', 'политик', 'законодательный', 'республиканский', 'собрание'],
    'sport': ['футбол', 'тренер', 'баскетбол', 'турнир', 'расписание', 'игры', 'лига', 'дивизион', 'команда', 'полузащитник'],
    'business': ['технология', 'платформа', 'компания', 'производство', 'продукт', 'дизайн',
                 'обновление', 'автоматизация', 'пользователь', 'эксплуатация'],
    'literature': ['книга', 'литература', 'художественная', 'толстой', 'шекспир', 'достоевский', 'роман', 'фанфин',
                   'приключение'],
    'crime': ['полиция', 'суд', 'преступник', 'инцидент', 'штраф', 'преступление', 'тюрьма', 'расследование', 
              'расследовать', 'жертва'],
    'travel': ['турист', 'туризм', 'путешествие', 'море', 'каникулы', 'пляж', 'отель', 'хостел', 'отпуск'],
    'games': ['игры', 'xbox', 'игрок', 'steam', 'карты', 'игроки', 'урон', 'switch', 'дракон', 'персонаж', 
              'геймплей', 'консоль'],
    'arhitecture': ['здание', 'построить', 'церковь', 'строение', 'мост', 
                    'крыша', 'улица', 'стиль', 'башня', 'спроектирован', 'камень',
                    'архитектура']
}

In [3]:
big_df_ru = pd.read_csv('/home/mlepekhin/data/big/ru.csv')
big_df_ru.head()

,Unnamed: 0,target,text
0,0,A8,"СИАБ раскидывает розничную сеть , Деловой Пете..."
1,1,A1,Мужчины о женщинах и женщины о мужчинах . Цен...
2,2,A8,Сервисное обслуживание и ремонт « Ивановская ...
3,3,A17,"Представляю вам коллекцию книг для блоггеров ,..."
4,4,A8,"Статьи и заметки Как мы уже сообщали , 26 ноя..."


In [4]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [5]:
def get_texts_for_topic_ru(topic_words, df, sent_for_topic):
    result = []
    
    for text in df.text.values[:10000]:
        prefix = ' '.join(text.split()[:100]).lower()
        doc = Doc(prefix)
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)
        for token in doc.tokens:
            token.lemmatize(morph_vocab)
            if token.lemma in topic_words:
                result.append(text)
                break
        if len(result) == sent_for_topic:
            return result
    return result


def make_topic_sentences_df(topic_dict, df, sent_for_topic=100, get_texts_for_topic=None):
    result_df = pd.DataFrame()
    
    for target in np.unique(df.target.values):
        genre_df = df[df.target == target]
        for topic, topic_words in tqdm.tqdm(topic_dict.items()):
            for sentence in get_texts_for_topic(set(topic_words), genre_df, sent_for_topic):
                result_df = result_df.append(
                    {'target': target, 'topic': topic, 'text': sentence},
                    ignore_index=True
                )
    return result_df

In [6]:
sent_df_ru = make_topic_sentences_df(ru_topics, big_df_ru, get_texts_for_topic=get_texts_for_topic_ru)

100%|██████████| 10/10 [02:51<00:00, 17.13s/it]


In [19]:
print(sent_df_ru.shape)
#print(sent_df_ru.sentence.values[:5])
sent_df_ru.head()

(8389, 3)


,target,text,topic
0,A1,Сайт посвящается светлой памяти гениального Жа...,music
1,A1,чуваки из Mastodon приготовили всем фанам и лю...,music
2,A1,"Если Вам не нравится безудержный , молодой Спр...",music
3,A1,"Сразу два дня – 26 и 27 сентября , в центральн...",music
4,A1,"Творчество и медитация Творчество – это , пре...",music


In [8]:
!mkdir '/home/mlepekhin/data/smart_genre'
!mkdir -p '/home/mlepekhin/data/smart_genre/ru'
!mkdir -p '/home/mlepekhin/data/smart_genre/en'

mkdir: cannot create directory ‘/home/mlepekhin/data/smart_genre’: File exists


In [6]:
def split_by_genres(df, result_dir):
    result_dict = {}
    
    for text, target in zip(df['text'].values, df['target'].values):
        if target not in result_dict:
            result_dict[target] = []
        result_dict[target].append(text)
        
    for key, value in result_dict.items():
        with open(pathjoin(result_dir, f'{key}.txt'), 'w') as fout:
            for text in value:
                fout.write(f'{text}\n')

In [7]:
#split_by_genres(sent_df_ru, '/home/mlepekhin/data/smart_genre/ru')

In [11]:
!ls '/home/mlepekhin/data/smart_genre/ru'
!wc -l '/home/mlepekhin/data/smart_genre/ru/A1.txt'

A11.txt  A14.txt  A17.txt  A4.txt  A8.txt  all.csv
A12.txt  A16.txt  A1.txt   A7.txt  A9.txt
2000 /home/mlepekhin/data/smart_genre/ru/A1.txt


In [12]:
sent_df_ru.to_csv('/home/mlepekhin/data/smart_genre/ru/all.csv')

## English

In [8]:
big_df_en = pd.read_csv('/home/mlepekhin/data/big/en.csv')
big_df_en.head()

,Unnamed: 0,target,text
0,0,A17,The ' Baronet Supreme ' is upholstered using ...
1,1,A17,This book explores some of the remarkable sto...
2,2,A17,Course Aims This course will teach support pr...
3,3,A7,In the News 24 November 2002 The Office of Fa...
4,4,A7,Signing the metadata for the SDSS federation ...


In [9]:
en_topics = {
    'music': ['music', 'band', 'album', 'songs', 'sound', 'love', 'rock', 'playing', 'guitar', 'jazz'],
    'education': ['student', 'learning', 'courses', 'teaching', 'skills', 'education', 'study', 'college'],
    'politics': ['election', 'votes', 'assembly', 'candidate', 'democratic', 
                 'council', 'minister', 'parliament', 'politician', 'legislative', 'seats', 'vote'],
    'sport': ['football', 'coach', 'basketball', 'tournament', 'schedule', 'games', 'league', 'division', 'team', 'teams'],
    'business': ['technology', 'platform', 'companies', 'industry', 'product', 'design',
                 'upgrade', 'automation', 'users', 'ideas'],
    'literature': ['books', 'literature', 'fiction', 'tolstoy', 'shakespeare', 'dostoevsky', 'romanism', 'fanfics', 'adventure'],
    'crime': ['police', 'court', 'officer', 'incident', 'charges', 'crime', 'prison', 'investigation', 'accused', 'victim'],
    'travel': ['tourist', 'tourism', 'travel', 'seaside', 'vacation', 'beach', 'hotel', 'hostel'],
    'games': ['games', 'xbox', 'players', 'steam', 'cards', 'player', 'damage', 'switch', 'dragon', 'character', 
              'reload', 'console'],
    'arhitecture': ['building', 'historic', 'church', 'buildings', 'brick', 
                    'roof', 'street', 'style', 'tower', 'designed', 'stone', 'architecture']
}

In [10]:
import nltk
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 

lemmatizer = WordNetLemmatizer() 

print("rocks :", lemmatizer.lemmatize("rocks")) 
print("corpora :", lemmatizer.lemmatize("corpora")) 

rocks : rock
corpora : corpus


In [11]:
def get_texts_for_topic_en(topic_words, df, sent_for_topic):
    result = []
    
    for text in df.text.values[:10000]:
        prefix_tokens = text.lower().split()[:100]
        for token in prefix_tokens:
            if lemmatizer.lemmatize(token) in topic_words:
                result.append(text)
                break
        if len(result) == sent_for_topic:
            return result
    return result

In [12]:
sent_df_en = make_topic_sentences_df(en_topics, big_df_en, get_texts_for_topic=get_texts_for_topic_en)

100%|██████████| 10/10 [00:13<00:00,  1.36s/it]


In [13]:
print(sent_df_en.shape)
sent_df_en.head()

(10842, 3)


,target,text,topic
0,A1,At Upton Heath School we aim to give the chil...,music
1,A1,VR in Education is a research project investi...,music
2,A1,"Like the label , we did it because we could ...",music
3,A1,""" For you are all sons of God through faith i...",music
4,A1,Provides an overview of abortion law in the U...,music


In [16]:
sent_df_en.to_csv('/home/mlepekhin/data/smart_genre/en/all.csv')

In [20]:
#split_by_genres(sent_df_en, '/home/mlepekhin/data/smart_genre/en')

In [21]:
!ls '/home/mlepekhin/data/smart_genre/en'
!wc -l '/home/mlepekhin/data/smart_genre/en/A1.txt'

A11.txt  A14.txt  A17.txt  A22.txt  A7.txt  A9.txt
A12.txt  A16.txt  A1.txt   A4.txt   A8.txt
1994 /home/mlepekhin/data/smart_genre/en/A1.txt


In [1]:
!ls '/home/mlepekhin/data/smart_genre/ru/all.csv'

A11.txt  A14.txt  A17.txt  A4.txt  A8.txt  all.csv
A12.txt  A16.txt  A1.txt   A7.txt  A9.txt
